**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Start

Deployment and Service

https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/

In [1]:
kubectl get pods -A

NAMESPACE     NAME                               READY   STATUS    RESTARTS      AGE
kube-system   coredns-78fcd69978-n5jfj           1/1     Running   0             71m
kube-system   etcd-minikube                      1/1     Running   0             71m
kube-system   kube-apiserver-minikube            1/1     Running   0             71m
kube-system   kube-controller-manager-minikube   1/1     Running   0             71m
kube-system   kube-proxy-bffmk                   1/1     Running   0             71m
kube-system   kube-scheduler-minikube            1/1     Running   0             71m
kube-system   storage-provisioner                1/1     Running   1 (70m ago)   71m


# Warning

**we install software from an unknown source**

In [2]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

deployment.apps/my-nginx created


In [3]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

service/my-nginx created


In [4]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-97g4w   1/1     Running   0          23s
pod/my-nginx-5b56ccd65f-mhqpf   1/1     Running   0          23s

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   75m
service/my-nginx     ClusterIP   10.109.82.109   <none>        80/TCP    11s

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   2/2     2            2           23s

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   2         2         2       23s


In [5]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

apiVersion: v1
kind: Service
metadata:
  name: my-nginx
  labels:
    run: my-nginx
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: my-nginx


In [6]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 2
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80



In [7]:
kubectl get service my-nginx

NAME       TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
my-nginx   ClusterIP   10.109.82.109   <none>        80/TCP    3m18s


In [8]:
kubectl get service my-nginx -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2022-01-18T10:41:07Z"
  labels:
    run: my-nginx
  name: my-nginx
  namespace: default
  resourceVersion: "3651"
  uid: fabd50ab-d4f7-43ce-92d3-dc6ab613ef0b
spec:
  clusterIP: 10.109.82.109
  clusterIPs:
  - 10.109.82.109
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    run: my-nginx
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [10]:
kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}'

10.109.82.109

In [11]:
IP=$(kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}')
curl $IP

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [12]:
echo http://$IP

http://10.109.82.109


In [13]:
kubectl get service -A -o jsonpath='{.items[*].spec.clusterIP}'

10.96.0.1 10.109.82.109 10.96.0.10

In [16]:
kubectl get pods --all-namespaces -o go-template \
--template="{{range .items}}{{.metadata.namespace}}/{{.metadata.name}}:{{println}}{{range .spec.containers}}  {{.image}}:{{.securityContext}}
{{end}}{{end}}" | grep --color=always -E '(^|privileged|NET_BIND_SERVICE)'


default/my-nginx-5b56ccd65f-97g4w:
  nginx:<no value>
default/my-nginx-5b56ccd65f-mhqpf:
  nginx:<no value>
kube-system/coredns-78fcd69978-n5jfj:
  k8s.gcr.io/coredns/coredns:v1.8.4:map[allowPrivilegeEscalation:false capabilities:map[add:[NET_BIND_SERVICE] drop:[all]] readOnlyRootFilesystem:true]
kube-system/etcd-minikube:
  k8s.gcr.io/etcd:3.5.0-0:<no value>
kube-system/kube-apiserver-minikube:
  k8s.gcr.io/kube-apiserver:v1.22.2:<no value>
kube-system/kube-controller-manager-minikube:
  k8s.gcr.io/kube-controller-manager:v1.22.2:<no value>
kube-system/kube-proxy-bffmk:
  k8s.gcr.io/kube-proxy:v1.22.2:map[privileged:true]
kube-system/kube-scheduler-minikube:
  k8s.gcr.io/kube-scheduler:v1.22.2:<no value>
kube-system/storage-provisioner:
  gcr.io/k8s-minikube/storage-provisioner:v5:<no value>


In [17]:
kubectl get pods -o yaml | grep image:

    - image: nginx
      image: nginx:latest
    - image: nginx
      image: nginx:latest


In [18]:
kubectl get pods -A -o yaml | grep image:

    - image: nginx
      image: nginx:latest
    - image: nginx
      image: nginx:latest
      image: k8s.gcr.io/coredns/coredns:v1.8.4
      image: k8s.gcr.io/coredns/coredns:v1.8.4
      image: k8s.gcr.io/etcd:3.5.0-0
      image: k8s.gcr.io/etcd:3.5.0-0
      image: k8s.gcr.io/kube-apiserver:v1.22.2
      image: k8s.gcr.io/kube-apiserver:v1.22.2
      image: k8s.gcr.io/kube-controller-manager:v1.22.2
      image: k8s.gcr.io/kube-controller-manager:v1.22.2
      image: k8s.gcr.io/kube-proxy:v1.22.2
      image: k8s.gcr.io/kube-proxy:v1.22.2
      image: k8s.gcr.io/kube-scheduler:v1.22.2
      image: k8s.gcr.io/kube-scheduler:v1.22.2
      image: gcr.io/k8s-minikube/storage-provisioner:v5
      image: gcr.io/k8s-minikube/storage-provisioner:v5


In [19]:
ps aux |grep python

root         221  0.2  0.1 489804 83160 ?        Ssl  09:25   0:34 /usr/bin/python3 /usr/local/bin/jupyter-lab --ip 192.168.49.2 --allow-root --log-level=INFO
root        6920  0.0  0.0 698028 55496 ?        Ssl  09:41   0:01 /usr/bin/python3 -m bash_kernel -f /root/.local/share/jupyter/runtime/kernel-4e617b21-ef6b-484c-ab07-8adf1cf24f6b.json
root        6934  0.0  0.0   6004  3000 pts/1    Ss+  09:41   0:00 /usr/bin/bash --rcfile /usr/local/lib/python3.8/dist-packages/pexpect/bashrc.sh
root       14193  0.0  0.0 698028 55492 ?        Ssl  10:13   0:01 /usr/bin/python3 -m bash_kernel -f /root/.local/share/jupyter/runtime/kernel-d887ea8e-2664-4248-9037-14b00bf978fd.json
root       14207  0.0  0.0   6000  3212 pts/3    Ss+  10:13   0:00 /usr/bin/bash --rcfile /usr/local/lib/python3.8/dist-packages/pexpect/bashrc.sh
root       14500  0.0  0.0 698012 55392 ?        Ssl  10:14   0:00 /usr/bin/python3 -m bash_kernel -f /root/.local/share/jupyter/runtime/kernel-5ad7218f-4f56-44d2-854c-07870d3

In [21]:
kubectl get pod -n kube-system coredns-78fcd69978-n5jfj -o yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: "2022-01-18T09:25:40Z"
  generateName: coredns-78fcd69978-
  labels:
    k8s-app: kube-dns
    pod-template-hash: 78fcd69978
  name: coredns-78fcd69978-n5jfj
  namespace: kube-system
  ownerReferences:
  - apiVersion: apps/v1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: coredns-78fcd69978
    uid: 22c45d16-a872-43b5-8d47-4aa714446424
  resourceVersion: "459"
  uid: 618a81e6-1cba-41e6-a8ea-758a11af7b74
spec:
  containers:
  - args:
    - -conf
    - /etc/coredns/Corefile
    image: k8s.gcr.io/coredns/coredns:v1.8.4
    imagePullPolicy: IfNotPresent
    livenessProbe:
      failureThreshold: 5
      httpGet:
        path: /health
        port: 8080
        scheme: HTTP
      initialDelaySeconds: 60
      periodSeconds: 10
      successThreshold: 1
      timeoutSeconds: 5
    name: coredns
    ports:
    - containerPort: 53
      name: dns
      protocol: UDP
    - containerPort: 53
      name: dns